In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Regresión Lineal").getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [5]:
datos=spark.read.csv("tienda.csv",header=True,inferSchema=True)

In [6]:
datos.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
datos.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [9]:
datos.head(2)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005),
 Row(Email='hduke@hotmail.com', Address='4547 Archer CommonDiazchester, CA 06566-8576', Avatar='DarkGreen', Avg Session Length=31.92627202636016, Time on App=11.109460728682564, Time on Website=37.268958868297744, Length of Membership=2.66403418213262, Yearly Amount Spent=392.2049334443264)]

In [10]:
datos.count()

500

In [11]:
datos.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [12]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [13]:
datos.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [16]:
constructor=VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership']
                            ,outputCol='features')

In [18]:
salida=constructor.transform(datos)

In [20]:
salida.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [22]:
salida.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [23]:
datosFinales=salida.select("features","Yearly Amount Spent")

In [24]:
datosFinales.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [25]:
entrena,evalua=datosFinales.randomSplit([0.7,0.3])

In [27]:
entrena.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                359|
|   mean|  498.0692705343655|
| stddev|  76.45797938291338|
|    min|   266.086340948469|
|    max|  744.2218671047146|
+-------+-------------------+



In [29]:
evalua.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                141|
|   mean| 502.48334047842667|
| stddev|    86.369200618166|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [30]:
lr=LinearRegression(labelCol="Yearly Amount Spent")

In [31]:
modeloLR=lr.fit(entrena)

In [32]:
resEvaluacion=modeloLR.evaluate(evalua)

In [34]:
resEvaluacion.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  12.462611514317814|
|  -3.961242165422675|
|  12.552927247486537|
|   8.821708124273528|
|   4.890256474105513|
|  11.319031919891813|
|  1.0546968953008218|
| -5.1401718749150405|
|  7.3684184391190115|
|-0.02662709266053298|
|  -4.624227380987065|
|  -9.478292288397142|
| -2.2281774049858996|
| -16.416815622883234|
| -0.8711090395483438|
|   8.036754966713886|
|  -1.665381569600413|
|-0.46339795798473915|
| -16.169647597102596|
|  13.201646366069667|
+--------------------+
only showing top 20 rows



In [35]:
resEvaluacion.rootMeanSquaredError

10.086279114041155

In [36]:
resEvaluacion.r2

0.9862647919858295

In [38]:
resEvaluacion.predictions.show()

+--------------------+-------------------+------------------+
|            features|Yearly Amount Spent|        prediction|
+--------------------+-------------------+------------------+
|[29.5324289670579...|  408.6403510726275|396.17773955830967|
|[30.4925366965402...|  282.4712457199145| 286.4324878853372|
|[30.7377203726281...|  461.7807421962299|449.22781494874334|
|[30.9716756438877...|  494.6386097568927| 485.8169016326192|
|[31.0472221394875...|  392.4973991890214| 387.6071427149159|
|[31.1695067987115...|  427.3565308022928|  416.037498882401|
|[31.3895854806643...|  410.0696110599829|409.01491416468207|
|[31.5147378578019...|  489.8124879964614|494.95265987137645|
|[31.6548096756927...|  475.2634237275485| 467.8950052884295|
|[31.6610498227460...| 416.35835357990084|416.38498067256137|
|[31.7242025238451...|  503.3878872879605|508.01211466894756|
|[31.8093003166791...|  536.7718993628412| 546.2501916512383|
|[31.8124825597242...|  392.8103449837972| 395.0385223887831|
|[31.816